<a href="https://colab.research.google.com/github/Itsuki-Hamano123/dim-compression/blob/master/experiments/XGBoost/pure_xgb_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 次元圧縮を行わずに分類

In [1]:
import os
import datetime

import cloudpickle
import pandas as pd

from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb

In [2]:
print('{lib}:{ver}'.format(lib='xgboost',ver=xgb.__version__))
!pip show scikit-learn

xgboost:0.90
Name: scikit-learn
Version: 0.22.2.post1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: joblib, numpy, scipy
Required-by: yellowbrick, umap-learn, textgenrnn, sklearn, sklearn-pandas, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute


# covtypeデータ読み込み
特徴量54次元で7クラス分類を行うデータ<br>
[scikit-learnのリファレンス](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype)<br>
[データセットの詳細ページ](https://archive.ics.uci.edu/ml/datasets/Covertype)


In [3]:
SEED = 2

data_bunch = fetch_covtype(random_state=SEED,
                           shuffle=True,
                           return_X_y=False)

print('data shape:{shape}'.format(shape=data_bunch.data.shape))
print('target shape:{shape}'.format(shape=data_bunch.target.shape))
data_bunch

data shape:(581012, 54)
target shape:(581012,)


{'DESCR': ".. _covtype_dataset:\n\nForest covertypes\n-----------------\n\nThe samples in this dataset correspond to 30×30m patches of forest in the US,\ncollected for the task of predicting each patch's cover type,\ni.e. the dominant species of tree.\nThere are seven covertypes, making this a multiclass classification problem.\nEach sample has 54 features, described on the\n`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.\nSome of the features are boolean indicators,\nwhile others are discrete or continuous measurements.\n\n**Data Set Characteristics:**\n\n    =================   ============\n    Classes                        7\n    Samples total             581012\n    Dimensionality                54\n    Features                     int\n    =================   ============\n\n:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;\nit returns a dictionary-like object\nwith the feature matrix in the ``data`` member\nand the target values

In [4]:
# 訓練データ,テストデータに分割
train_X, test_X, train_y, test_y = train_test_split(data_bunch.data, data_bunch.target,
                                                    test_size=0.2, random_state=SEED)
# 訓練データ,検証データに分割
train_X, validation_X, train_y, validation_y = train_test_split(train_X, train_y,
                                                                test_size=0.2, random_state=SEED)

print('train size:{size}'.format(size=train_y.shape[0]))
print('validation size:{size}'.format(size=validation_y.shape[0]))
print('test size:{size}'.format(size=test_y.shape[0]))

train size:371847
validation size:92962
test size:116203


In [5]:
NUM_CLASS = 7

# データの前処理

## 標準化

In [6]:
STD_SCALER = StandardScaler()
# 標準化器を作成
STD_SCALER.fit(train_X)
print('sclaer fit end')

sclaer fit end


In [7]:
# 訓練データを標準化
std_train_X = STD_SCALER.transform(train_X, copy=True)
std_train_X[:,0]

array([ 0.31952626, -1.6259504 ,  1.09557117, ..., -0.64605958,
       -0.18830037,  1.2529259 ])

In [8]:
# 検証/テストデータも標準化
std_validation_X = STD_SCALER.transform(validation_X, copy=True)
std_test_X = STD_SCALER.transform(test_X, copy=True)

# XGBosot(classifier)でクラス分類モデル構築

In [9]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [10]:
cd My Drive/機械学習練習/次元圧縮手法_比較/experiment/XGBoost

/gdrive/My Drive/機械学習練習/次元圧縮手法_比較/experiment/XGBoost


In [11]:
!ls 

 compress_dim_ica.ipynb  'compress_dim_pca.ipynb のコピー'
 compress_dim_pca.ipynb   pure_xgb_cls.ipynb


In [12]:
LOG_DIR = '../logs'
model_name = 'pure_xgboost'
xgb.XGBClassifier()
xgb_cls = xgb.XGBRFClassifier(random_state=SEED, tree_method='gpu_hist', gpu_id=0)

## モデルの訓練

In [14]:
xgb_cls.fit(X=std_train_X, y=train_y-1)

XGBRFClassifier(base_score=0.5, colsample_bylevel=1, colsample_bynode=0.8,
                colsample_bytree=1, gamma=0, gpu_id=0, learning_rate=1,
                max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
                n_estimators=100, n_jobs=1, nthread=None,
                objective='multi:softprob', random_state=2, reg_alpha=0,
                reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
                subsample=0.8, tree_method='gpu_hist', verbosity=1)

## モデルの評価

In [16]:
train_score = xgb_cls.score(X=std_train_X, y=train_y-1)
test_score = xgb_cls.score(X=std_test_X, y=test_y-1)
print('train score:{score}'.format(score=train_score))
print('test score:{score}'.format(score=test_score))

train score:0.6889849857602723
test score:0.6910751013312909


### 主な分類指標を示すレポート

In [17]:
predict_y = xgb_cls.predict(data=std_test_X)

report = classification_report(y_true=test_y, y_pred=predict_y+1)
print(report)

              precision    recall  f1-score   support

           1       0.66      0.71      0.69     42488
           2       0.72      0.76      0.74     56473
           3       0.63      0.72      0.67      7355
           4       0.68      0.27      0.39       524
           5       0.00      0.00      0.00      1866
           6       0.50      0.00      0.00      3432
           7       0.74      0.41      0.53      4065

    accuracy                           0.69    116203
   macro avg       0.56      0.41      0.43    116203
weighted avg       0.68      0.69      0.67    116203



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 混同行列

In [18]:
cm = confusion_matrix(y_true=test_y, y_pred=predict_y+1)
print(cm)

[[30333 11632     1     0     0     0   522]
 [13044 42856   508     4     0     2    59]
 [    0  2006  5306    43     0     0     0]
 [    0     0   380   144     0     0     0]
 [    0  1864     2     0     0     0     0]
 [    0  1147  2263    20     0     2     0]
 [ 2384    17     0     0     0     0  1664]]


### モデルの保存

In [19]:
model_file = os.path.join(LOG_DIR, model_name+'_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.pkl')

with open(model_file, 'wb') as f:
      f.write(cloudpickle.dumps(xgb_cls))

In [20]:
!ls ../logs/

compress_dim_ica_20200706-000503.h5  ica_xgboost_20200712-055216.pkl
compress_dim_pca_20200705-080246.h5  ica_xgboost.pkl
compress_dim_pca_20200705-080838.h5  pca_xgboost_20200712-055438.pkl
compress_dim_pca_20200706-122832.h5  pure_xgboost_20200712-060727.pkl
fit
